In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv("Network_data.csv", sep=";", decimal=",")
df = df.dropna() 


# the head rows of the dataframe.

In [ ]:
df.head()

# Encoder : 

In [3]:
from sklearn import preprocessing

# Apply Leabl encoding
le = preprocessing.LabelEncoder()
le.fit(df['IPV4_SRC_ADDR'])
df['IPV4_SRC_ADDR']=le.transform(df['IPV4_SRC_ADDR'])

le.fit(df['IPV4_DST_ADDR'])
df['IPV4_DST_ADDR']=le.transform(df['IPV4_DST_ADDR'])

from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoding
encoder = OneHotEncoder()
encoded = encoder.fit_transform(df[['PROTOCOL_MAP']])
encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names_out(['PROTOCOL_MAP']))
final_df = df.join(encoded_df)
final_df.drop('PROTOCOL_MAP', axis=1, inplace=True)

# fill 0 in one-hot encoding
final_df['PROTOCOL_MAP_udp'] = final_df['PROTOCOL_MAP_udp'].fillna(0)
final_df['PROTOCOL_MAP_tcp'] = final_df['PROTOCOL_MAP_tcp'].fillna(0)
final_df['PROTOCOL_MAP_gre'] = final_df['PROTOCOL_MAP_gre'].fillna(0)
final_df['PROTOCOL_MAP_icmp'] = final_df['PROTOCOL_MAP_icmp'].fillna(0)
final_df['PROTOCOL_MAP_ipv6-icmp'] = final_df['PROTOCOL_MAP_ipv6-icmp'].fillna(0)


,FLOW_ID,L4_SRC_PORT,IPV4_SRC_ADDR,L4_DST_PORT,IPV4_DST_ADDR,FIRST_SWITCHED,FLOW_DURATION_MILLISECONDS,LAST_SWITCHED,PROTOCOL,TCP_FLAGS,...,OUT_PKTS,ANALYSIS_TIMESTAMP,ANOMALY,ALERT,ID,PROTOCOL_MAP_gre,PROTOCOL_MAP_icmp,PROTOCOL_MAP_ipv6-icmp,PROTOCOL_MAP_tcp,PROTOCOL_MAP_udp
0,372018590,53950,93,53,70,1647760833,1,1647760833,17,0,...,2,1647760873,0.0,None,0,0.0,0.0,0.0,0.0,1.0
1,368604472,37914,105,38303,43,1647686725,1,1647686725,6,22,...,1,1647687338,0.0,Port Scanning,1,0.0,0.0,0.0,1.0,0.0
2,367513992,33216,105,18757,33,1647680044,0,1647680044,6,22,...,1,1647680115,1.0,Port Scanning,2,0.0,0.0,0.0,1.0,0.0
3,371997670,48627,53,53,70,1647760172,0,1647760172,17,0,...,2,1647760207,0.0,None,3,0.0,0.0,0.0,0.0,1.0
4,374311119,35939,53,53,70,1647780916,0,1647780916,17,0,...,2,1647780984,0.0,None,4,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213716,369777076,9090,5633,22,63,1647697117,0,1647697117,6,22,...,1,1647697169,0.0,None,213716,0.0,0.0,0.0,0.0,0.0
213718,369951971,48050,50,443,4780,1647700921,110025,1647701031,6,24,...,13,1647701099,0.0,None,213718,0.0,0.0,0.0,0.0,0.0
213720,374268311,54898,7095,16411,45,1647779778,51,1647779778,6,22,...,1,1647779832,0.0,None,213720,0.0,0.0,0.0,0.0,0.0
213721,369608959,48872,54,443,2117,1647693306,205,1647693306,6,27,...,10,1647693357,0.0,None,213721,0.0,0.0,0.0,0.0,0.0


# Split the dataset into Train and Test sets with test_size=30% and random_state=23

In [4]:
from sklearn.model_selection import train_test_split
X = final_df.drop('ALERT',axis=1)
y = final_df['ALERT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30 , random_state=11)

# DecisionTree

In [5]:
from sklearn.metrics import confusion_matrix, accuracy_score , ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)
predictions = dtree.predict(X_test)

print("--Decision Trees--\n")
print(" confusion matrix : \n",confusion_matrix(y_test,predictions) , "\n")
print("accuracy : ",accuracy_score(y_test,predictions)*100)

--Decision Trees--

 confusion matrix : 
 [[    8     0     0]
 [    0 27528     0]
 [    0     1  7615]] 

accuracy :  99.99715521165226


# GaussianNB

In [7]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
 
y_pred = gnb.predict(X_test)

print("--Naive Bayes Classifiers--\n")
print(" confusion matrix : \n",confusion_matrix(y_test,y_pred) , "\n")
print("accuracy : ",accuracy_score(y_test,y_pred)*100)

--Naive Bayes Classifiers--

 confusion matrix : 
 [[    8     0     0]
 [    0 27508    20]
 [    0     5  7611]] 

accuracy :  99.92888029130633


# RandomForest

In [6]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators = 20, criterion = 'gini', max_depth = 4) 
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

print("--Random Forests--\n")
print(" confusion matrix : \n",confusion_matrix(y_test,rfc_pred) , "\n")
print("accuracy : ",accuracy_score(y_test,rfc_pred)*100)

--Random Forests--

 confusion matrix : 
 [[    8     0     0]
 [    0 27383   145]
 [    0     4  7612]] 

accuracy :  99.57612653618571


# Conclusion

### Since we need to classify network connections, we used Algorithm Classification, so we tried KNeighbour, GaussianNB, RandomForest, Support Vector Machine, and DecisionTree. We found the highest accuracy percentage in the algorithms, that is  DecisionTree , GaussianNB, Random Forests, ..Respectively